In [31]:
!pip install hvplot
!pip install plotly

In [32]:
# dependencies .. lets put all the dependencies here
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import numpy as np
import datetime as dt
import hvplot.pandas
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [33]:
df2 = pd.read_csv('credit_card_transactions.csv')
df2.head()

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,merch_zipcode
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,28705.0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,NaN
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,83236.0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,NaN
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,22844.0


In [34]:
df2["trans_date_trans_time"] = pd.to_datetime(df2["trans_date_trans_time"], utc=True)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 24 columns):
 #   Column                 Non-Null Count    Dtype              
---  ------                 --------------    -----              
 0   Unnamed: 0             1296675 non-null  int64              
 1   trans_date_trans_time  1296675 non-null  datetime64[ns, UTC]
 2   cc_num                 1296675 non-null  int64              
 3   merchant               1296675 non-null  object             
 4   category               1296675 non-null  object             
 5   amt                    1296675 non-null  float64            
 6   first                  1296675 non-null  object             
 7   last                   1296675 non-null  object             
 8   gender                 1296675 non-null  object             
 9   street                 1296675 non-null  object             
 10  city                   1296675 non-null  object             
 11  state                  1

In [35]:
data = df2
df2 = data.copy()
df2['trans_date'] = pd.to_datetime(df2['trans_date_trans_time'], utc=True).dt.date
df2['trans_year'] = pd.to_datetime(df2['trans_date_trans_time'], utc=True).dt.year
df2['trans_month'] = pd.to_datetime(df2['trans_date_trans_time'], utc=True).dt.month
df2['trans_day'] = pd.to_datetime(df2['trans_date_trans_time'], utc=True).dt.day
df2


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,trans_num,unix_time,merch_lat,merch_long,is_fraud,merch_zipcode,trans_date,trans_year,trans_month,trans_day
0,0,2019-01-01 00:00:18+00:00,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,28705.0,2019-01-01,2019,1,1
1,1,2019-01-01 00:00:44+00:00,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,NaN,2019-01-01,2019,1,1
2,2,2019-01-01 00:00:51+00:00,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,83236.0,2019-01-01,2019,1,1
3,3,2019-01-01 00:01:16+00:00,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,NaN,2019-01-01,2019,1,1
4,4,2019-01-01 00:03:06+00:00,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,22844.0,2019-01-01,2019,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,1296670,2020-06-21 12:12:08+00:00,30263540414123,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,...,440b587732da4dc1a6395aba5fb41669,1371816728,36.841266,-111.690765,0,NaN,2020-06-21,2020,6,21
1296671,1296671,2020-06-21 12:12:19+00:00,6011149206456997,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,...,278000d2e0d2277d1de2f890067dcc0a,1371816739,38.906881,-78.246528,0,22630.0,2020-06-21,2020,6,21
1296672,1296672,2020-06-21 12:12:32+00:00,3514865930894695,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,...,483f52fe67fabef353d552c1e662974c,1371816752,33.619513,-105.130529,0,88351.0,2020-06-21,2020,6,21
1296673,1296673,2020-06-21 12:13:36+00:00,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,...,d667cdcbadaaed3da3f4020e83591c83,1371816816,42.788940,-103.241160,0,69367.0,2020-06-21,2020,6,21


In [67]:
# Step 1: Ensure 'dob' is a datetime object
df2['dob'] = pd.to_datetime(df2['dob'], errors='coerce')

# Step 2: Calculate the 'Age' column
df2['age'] = df2['trans_year'] - df2['dob'].dt.year

# Step 3: Define age groups based on the ranges provided
def age_description(age):
    if age < 18:
        return 'Age: 18 & Under / Youth / Teens'
    elif 19 <= age <= 28:
        return 'Age: 19-28 / Young Adults'
    elif 29 <= age <= 38:
        return 'Age: 29-38 / Early Career / Early Adults'
    elif 39 <= age <= 48:
        return 'Age: 39-48 / Mid-Life Professionals / Experienced Adults'
    elif 49 <= age <= 58:
        return 'Age: 49-58 / Seasoned Adults / Mature Professionals'
    elif 59 <= age <= 68:
        return 'Age: 59-68 / Pre-Retirement / Early Retirees'
    elif 69 <= age <= 78:
        return 'Age: 69-78 / Active Retirees / Golden Years'
    else:
        return 'Age: 79 & Older / Seniors / Elderly'

# Step 4: Create the 'Age Description' column
df2['age_description'] = df2['age'].apply(age_description)

# View the final dataframe with the new columns
df2[['age', 'age_description']].head()


,age,age_description
0,31,Age: 29-38 / Early Career / Early Adults
1,41,Age: 39-48 / Mid-Life Professionals / Experien...
2,57,Age: 49-58 / Seasoned Adults / Mature Professi...
3,52,Age: 49-58 / Seasoned Adults / Mature Professi...
4,33,Age: 29-38 / Early Career / Early Adults


In [68]:
# Count the number of people in each age group
age_group_counts = df2['age_description'].value_counts()

# Display the counts
print(age_group_counts)


age_description
Age: 29-38 / Early Career / Early Adults                    300943
Age: 39-48 / Mid-Life Professionals / Experienced Adults    300161
Age: 49-58 / Seasoned Adults / Mature Professionals         200411
Age: 19-28 / Young Adults                                   181766
Age: 59-68 / Pre-Retirement / Early Retirees                143773
Age: 69-78 / Active Retirees / Golden Years                  78761
Age: 79 & Older / Seniors / Elderly                          77430
Age: 18 & Under / Youth / Teens                              13430
Name: count, dtype: int64


In [180]:
import plotly.express as px

# Recalculate the age group counts
age_group_counts = df2['age_description'].value_counts().reset_index()

# Rename columns for clarity
age_group_counts.columns = ['age_group', 'count']

# Step 3: Create a pie chart using plotly.express
fig = px.pie(
    age_group_spending, 
    names='age_group', 
    values='total_spent', 
    title='Percentage of Total Transactions Spent by Each Age Group',
    color_discrete_sequence=px.colors.qualitative.Set3,
    height=500, width=800
)

# Add a pull parameter to "explode" the slices
fig.update_traces(pull=[0.05] * len(age_group_spending))  # Pull each slice out by 5%

# Update layout for centered title and pie chart
fig.update_layout(
    title={
        'text': 'Percentage of Total Transactions Spent by Each Age Group',
        'y': 0.85,  # position the title closer to the top
        'x': 0.5,   # center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'yanchor': 'middle',  # vertical centering
        'y': 0.5,             # legend is vertically centered
        'xanchor': 'left',     # move the legend to the right side
        'x': 1.3              # move the legend further right to avoid overlapping the pie chart
    },
    margin=dict(l=100)  # adds 100px left margin to move the pie chart to the left
)

# Display the pie chart
fig.show()


# Display the pie chart
fig.show()




In [179]:
import plotly.express as px

# Step 1: Group the data by 'age_description' and sum the 'amt' for each group
age_group_spending = df2.groupby('age_description')['amt'].sum().reset_index()

# Step 2: Rename columns for clarity
age_group_spending.columns = ['age_group', 'total_spent']

# Step 3: Create a pie chart using plotly.express
fig = px.pie(
    age_group_spending, 
    names='age_group', 
    values='total_spent', 
    title='Percentage of Total Transactions Spent by Each Age Group',
    color_discrete_sequence=px.colors.qualitative.Set3,
    height=500, width=800
)

# Add a pull parameter to "explode" the slices
fig.update_traces(pull=[0.05] * len(age_group_spending))  # Pull each slice out by 5%

# Update layout for centered title and pie chart
fig.update_layout(
    title={
        'text': 'Percentage of Total Transactions Spent by Each Age Group',
        'y': 0.85,  # position the title closer to the top
        'x': 0.5,   # center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'yanchor': 'middle',  # vertical centering
        'y': 0.5,             # legend is vertically centered
        'xanchor': 'left',     # move the legend to the right side
        'x': 1.3              # move the legend further right to avoid overlapping the pie chart
    },
    margin=dict(l=100)  # adds 100px left margin to move the pie chart to the left
)

# Display the pie chart
fig.show()



In [71]:
import pandas as pd
import plotly.express as px

# Step 1: Group by 'age_description' and 'category', then sum the amount spent
grouped = df2.groupby(['age_description', 'category'])['amt'].sum().reset_index()

# Step 2: For each age group, sort the spending by category and take the top 3
top_3_categories = grouped.groupby('age_description').apply(
    lambda x: x.nlargest(3, 'amt')
).reset_index(drop=True)

# Step 3: Create a bar chart using plotly
fig = px.bar(
    top_3_categories, x='age_description', y='amt', color='category', 
    title="Top 3 Spending Categories by Age Group",
    labels={'amt': 'Total Spend', 'age_description': 'Age Group'},
    height=500, width=800
)

fig.show()



C:\Users\reddo\AppData\Local\Temp\ipykernel_36932\2318994797.py:8: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [72]:
import plotly.express as px

# Step 1: Filter the data for the 'Travel' category
travel_spending = df2[df2['category'] == 'travel'].groupby('age_description')['amt'].sum().reset_index()

# Step 2: Create a bar chart using plotly for Travel spending
fig = px.bar(
    travel_spending, x='age_description', y='amt', 
    title="Total Spending on Travel by Age Group",
    labels={'amt': 'Total Spend on Travel', 'age_description': 'Age Group'},
    height=500, width=800
)

fig.show()


In [177]:
import plotly.express as px

# Step 1: Calculate total spending per age group
total_spending_by_age = df2.groupby('age_description')['amt'].sum().reset_index()

# Step 2: Calculate travel spending per age group
travel_spending_by_age = df2[df2['category'] == 'travel'].groupby('age_description')['amt'].sum().reset_index()

# Step 3: Merge both dataframes to calculate the percentage of travel spending
spending_comparison = pd.merge(travel_spending_by_age, total_spending_by_age, on='age_description', suffixes=('_travel', '_total'))

# Step 4: Calculate the percentage of travel spending for each age group
spending_comparison['travel_percentage'] = (spending_comparison['amt_travel'] / spending_comparison['amt_total']) * 100

# Step 5: Create a pie chart using plotly for the percentage of travel spending
fig = px.pie(
    spending_comparison, 
    values='travel_percentage', 
    names='age_description', 
    title="Percentage of Travel Spending as Part of Total Spending by Age Group",
    height=500, width=800
)

# Add a pull parameter to "explode" the slices
fig.update_traces(pull=[0.05] * len(spending_comparison))  # Pull each slice out by 5%

# Update layout to center title and adjust the pie chart position to the left
fig.update_layout(
    title={
        'text': "Percentage of Travel Spending as Part of Total Spending by Age Group",
        'y':0.85,  # position of the title from bottom to top (0-1 scale)
        'x':0.5,   # center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'yanchor': 'middle',  # vertical centering
        'y': 0.5,             # legend is vertically centered
        'xanchor': 'left',     # move the legend to the right side
        'x': 1.3              # move the legend further right to avoid overlapping the pie chart
    },
    margin=dict(l=100)  # adds 100px left margin to move the pie chart to the left
)

# Display the chart
fig.show()


In [186]:
import plotly.express as px

# Step 1: Filter data for the under-18 group
under_18_data = df2[df2['age_description'] == 'Age: 18 & Under / Youth / Teens']

# Step 2: Group the data by category and calculate total spend
under_18_spend_by_category = under_18_data.groupby('category')['amt'].sum().reset_index()

# Step 3: Calculate the percentage of total spend for each category
total_spend_under_18 = under_18_spend_by_category['amt'].sum()
under_18_spend_by_category['percentage'] = (under_18_spend_by_category['amt'] / total_spend_under_18) * 100

# Round percentage to one decimal place for display
under_18_spend_by_category['percentage'] = under_18_spend_by_category['percentage'].round(1)

# Keep 'amt' as numerical for plotting but create a formatted version for hover
under_18_spend_by_category['amt_formatted'] = under_18_spend_by_category['amt'].round(2).map('${:,.2f}'.format)

# Step 4: Create a pie chart using Plotly Express
fig = px.pie(
    under_18_spend_by_category, 
    names='category', 
    values='amt',  # Keep this numerical for plotting
    title='Age: 18 & Under / Youth / Teens: Spending by Category',
    hover_data=['amt_formatted'],  # Only show Total Spend on hover
    labels={'amt_formatted': 'Total Spend'}
)

# Customize the chart with percentages on the slices and display total spend on hover
fig.update_traces(
    textposition='inside', 
    textinfo='percent+label', 
    hovertemplate='Total Spend: $%{value:,.2f}', 
    pull=[0.05] * len(under_18_spend_by_category),  # Pull out each slice slightly
    marker=dict(line=dict(color='white', width=2))  # Add white border around each slice
    )

# Update the layout for better alignment of the pie chart and legend
fig.update_layout(
    width=800,  # Adjust width
    height=500,  # Adjust height
    title={
        'text': 'Age: 18 & Under / Youth / Teens: Spending by Category',
        'y':0.90,  # position of the title from bottom to top (0-1 scale)
        'x':0.56,   # center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'yanchor': 'middle',  # vertical centering
        'y': 0.5,             # legend is vertically centered
        'xanchor': 'left',     # move the legend to the right side
        'x': .8             # move the legend further right to avoid overlapping the pie chart
    },
    margin=dict(l=0  # adds 100px left margin to move the pie chart to the left
))
# Display the chart
fig.show()


In [175]:
# Filter the data for the 19-28 age group
young_adults_data = df2[df2['age_description'] == 'Age: 19-28 / Young Adults']

# Calculate the total spend by category for the 19-28 age group
young_adults_spending = young_adults_data.groupby('category')['amt'].sum().reset_index()

# Sort the values for a cleaner chart presentation
young_adults_spending = young_adults_spending.sort_values(by='amt', ascending=False)

# Create a pie chart using plotly
fig = px.pie(
    young_adults_spending, 
    names='category', 
    values='amt', 
    title='Age: 19-28 / Young Adults: Spending by Category', 
    hover_data={'amt': ':.2f'},
    labels={'amt': 'Total Spend'}
)

# Customize the chart with percentages on the slices and display total spend on hover
fig.update_traces(
    textposition='inside', 
    textinfo='percent+label', 
    hovertemplate='Total Spend: $%{value:,.2f}', 
    pull=[0.05] * len(young_adults_spending),  # Pull out each slice slightly
    marker=dict(line=dict(color='white', width=2))  # Add white border around each slice
)

# Update the layout for better alignment of the pie chart and legend
fig.update_layout(
    width=800,  # Adjust width
    height=500,  # Adjust height
    title={
        'text': 'Age: 19-28 / Young Adults: Spending by Category',
        'y':0.90,  # position of the title from bottom to top (0-1 scale)
        'x':0.56,   # center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'yanchor': 'middle',  # vertical centering
        'y': 0.5,             # legend is vertically centered
        'xanchor': 'left',     # move the legend to the right side
        'x': .8             # move the legend further right to avoid overlapping the pie chart
    },
    margin=dict(l=0  # adds 100px left margin to move the pie chart to the left
))
# Display the chart
fig.show()


In [185]:
# Filter the data for the 29-38 age group
early_career_data = df2[df2['age_description'] == 'Age: 29-38 / Early Career / Early Adults']

# Calculate the total spend by category for the 29-38 age group
early_career_spending = early_career_data.groupby('category')['amt'].sum().reset_index()

# Sort the values for a cleaner chart presentation
early_career_spending = early_career_spending.sort_values(by='amt', ascending=False)

# Create a pie chart using plotly
fig = px.pie(
    early_career_spending, 
    names='category', 
    values='amt', 
    title='Age: 29-38 / Early Career / Early Adults: Spending by Category', 
    hover_data={'amt': ':.2f'},
    labels={'amt': 'Total Spend'}
)

# Customize the chart with percentages on the slices and display total spend on hover
fig.update_traces(
    textposition='inside', 
    textinfo='percent+label', 
    hovertemplate='Total Spend: $%{value:,.2f}', 
    pull=[0.05] * len(early_career_spending),  # Pull out each slice slightly
    marker=dict(line=dict(color='white', width=2))  # Add white border around each slice
)

# Update the layout for better alignment of the pie chart and legend
fig.update_layout(
    width=800,  # Adjust width
    height=500,  # Adjust height
    title={
        'text': 'Age: 29-38 / Early Career / Early Adults: Spending by Category',
        'y':0.90,  # position of the title from bottom to top (0-1 scale)
        'x':0.56,   # center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'yanchor': 'middle',  # vertical centering
        'y': 0.5,             # legend is vertically centered
        'xanchor': 'left',     # move the legend to the right side
        'x': .8             # move the legend further right to avoid overlapping the pie chart
    },
    margin=dict(l=0  # adds 100px left margin to move the pie chart to the left
))
# Display the chart
fig.show()


In [184]:
# Filter the data for the 39-48 age group
mid_life_data = df2[df2['age_description'] == 'Age: 39-48 / Mid-Life Professionals / Experienced Adults']

# Calculate the total spend by category for the 39-48 age group
mid_life_spending = mid_life_data.groupby('category')['amt'].sum().reset_index()

# Sort the values for a cleaner chart presentation
mid_life_spending = mid_life_spending.sort_values(by='amt', ascending=False)

# Create a pie chart using plotly
fig = px.pie(
    mid_life_spending, 
    names='category', 
    values='amt', 
    title='Age: 39-48 / Mid-Life Professionals / Experienced Adults: Spending by Category', 
    hover_data={'amt': ':.2f'},
    labels={'amt': 'Total Spend'}
)

# Customize the chart with percentages on the slices and display total spend on hover
fig.update_traces(
    textposition='inside', 
    textinfo='percent+label', 
    hovertemplate='Total Spend: $%{value:,.2f}', 
    pull=[0.05] * len(mid_life_spending),  # Pull out each slice slightly
    marker=dict(line=dict(color='white', width=2))  # Add white border around each slice
)

# Update the layout for better alignment of the pie chart and legend
fig.update_layout(
    width=800,  # Adjust width
    height=500,  # Adjust height
    title={
        'text': 'Age: 39-48 / Mid-Life Professionals / Experienced Adults: Spending by Category',
        'y':0.90,  # position of the title from bottom to top (0-1 scale)
        'x':0.56,   # center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'yanchor': 'middle',  # vertical centering
        'y': 0.5,             # legend is vertically centered
        'xanchor': 'left',     # move the legend to the right side
        'x': .8             # move the legend further right to avoid overlapping the pie chart
    },
    margin=dict(l=0  # adds 100px left margin to move the pie chart to the left
))
# Display the chart
fig.show()


In [183]:
# Filter the data for the 49-58 age group
seasoned_adults_data = df2[df2['age_description'] == 'Age: 49-58 / Seasoned Adults / Mature Professionals']

# Calculate the total spend by category for the 49-58 age group
seasoned_adults_spending = seasoned_adults_data.groupby('category')['amt'].sum().reset_index()

# Sort the values for a cleaner chart presentation
seasoned_adults_spending = seasoned_adults_spending.sort_values(by='amt', ascending=False)

# Create a pie chart using plotly
fig = px.pie(
    seasoned_adults_spending, 
    names='category', 
    values='amt', 
    title='Age: 49-58 / Seasoned Adults / Mature Professionals: Spending by Category', 
    hover_data={'amt': ':.2f'},
    labels={'amt': 'Total Spend'}
)

# Customize the chart with percentages on the slices and display total spend on hover
fig.update_traces(
    textposition='inside', 
    textinfo='percent+label', 
    hovertemplate='Total Spend: $%{value:,.2f}', 
    pull=[0.05] * len(seasoned_adults_spending),  # Pull out each slice slightly
    marker=dict(line=dict(color='white', width=2))  # Add white border around each slice
)

# Update the layout for better alignment of the pie chart and legend
fig.update_layout(
    width=800,  # Adjust width
    height=500,  # Adjust height
    title={
        'text': 'Age: 49-58 / Seasoned Adults / Mature Professionals: Spending by Category', 
        'y':0.90,  # position of the title from bottom to top (0-1 scale)
        'x':0.56,   # center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'yanchor': 'middle',  # vertical centering
        'y': 0.5,             # legend is vertically centered
        'xanchor': 'left',     # move the legend to the right side
        'x': .8             # move the legend further right to avoid overlapping the pie chart
    },
    margin=dict(l=0  # adds 100px left margin to move the pie chart to the left
))
# Display the chart
fig.show()


In [182]:
# Filter the data for the 59-68 age group
early_retirees_data = df2[df2['age_description'] == 'Age: 59-68 / Pre-Retirement / Early Retirees']

# Calculate the total spend by category for the 59-68 age group
early_retirees_spending = early_retirees_data.groupby('category')['amt'].sum().reset_index()

# Sort the values for a cleaner chart presentation
early_retirees_spending = early_retirees_spending.sort_values(by='amt', ascending=False)

# Create a pie chart using plotly
fig = px.pie(
    early_retirees_spending, 
    names='category', 
    values='amt', 
    title='Age: 59-68 / Pre-Retirement / Early Retirees: Spending by Category', 
    hover_data={'amt': ':.2f'},
    labels={'amt': 'Total Spend'}
)

# Customize the chart with percentages on the slices and display total spend on hover
fig.update_traces(
    textposition='inside', 
    textinfo='percent+label', 
    hovertemplate='Total Spend: $%{value:,.2f}', 
    pull=[0.05] * len(early_retirees_spending),  # Pull out each slice slightly
    marker=dict(line=dict(color='white', width=2))  # Add white border around each slice
)

# Update the layout for better alignment of the pie chart and legend
fig.update_layout(
    width=800,  # Adjust width
    height=500,  # Adjust height
    title={
        'text': 'Age: 59-68 / Pre-Retirement / Early Retirees: Spending by Category',  
        'y':0.90,  # position of the title from bottom to top (0-1 scale)
        'x':0.56,   # center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'yanchor': 'middle',  # vertical centering
        'y': 0.5,             # legend is vertically centered
        'xanchor': 'left',     # move the legend to the right side
        'x': .8             # move the legend further right to avoid overlapping the pie chart
    },
    margin=dict(l=0  # adds 100px left margin to move the pie chart to the left
))
# Display the chart
fig.show()



In [181]:
# Filter the data for the 69-78 age group
active_retirees_data = df2[df2['age_description'] == 'Age: 69-78 / Active Retirees / Golden Years']

# Calculate the total spend by category for the 69-78 age group
active_retirees_spending = active_retirees_data.groupby('category')['amt'].sum().reset_index()

# Sort the values for a cleaner chart presentation
active_retirees_spending = active_retirees_spending.sort_values(by='amt', ascending=False)

# Create a pie chart using plotly
fig = px.pie(
    active_retirees_spending, 
    names='category', 
    values='amt', 
    title='Age: 69-78 / Active Retirees / Golden Years: Spending by Category', 
    hover_data={'amt': ':.2f'},
    labels={'amt': 'Total Spend'}
)

# Customize the chart with percentages on the slices and display total spend on hover
fig.update_traces(
    textposition='inside', 
    textinfo='percent+label', 
    hovertemplate='Total Spend: $%{value:,.2f}', 
    pull=[0.05] * len(active_retirees_spending),  # Pull out each slice slightly
    marker=dict(line=dict(color='white', width=2))  # Add white border around each slice
)

# Update the layout for better alignment of the pie chart and legend
fig.update_layout(
    width=800,  # Adjust width
    height=500,  # Adjust height
    title={
        'text': 'Age: 69-78 / Active Retirees / Golden Years: Spending by Category', 
        'y':0.90,  # position of the title from bottom to top (0-1 scale)
        'x':0.56,   # center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'yanchor': 'middle',  # vertical centering
        'y': 0.5,             # legend is vertically centered
        'xanchor': 'left',     # move the legend to the right side
        'x': .8             # move the legend further right to avoid overlapping the pie chart
    },
    margin=dict(l=0  # adds 100px left margin to move the pie chart to the left
))
# Display the chart
fig.show()

In [169]:
# Filter the data for the 79 & Older age group
seniors_data = df2[df2['age_description'] == 'Age: 79 & Older / Seniors / Elderly']

# Calculate the total spend by category for the 79 & Older age group
seniors_spending = seniors_data.groupby('category')['amt'].sum().reset_index()

# Sort the values for a cleaner chart presentation
seniors_spending = seniors_spending.sort_values(by='amt', ascending=False)

# Create a pie chart using plotly with adjusted size
fig = px.pie(
    seniors_spending, 
    names='category', 
    values='amt', 
    title='Age: 79 & Older / Seniors / Elderly: Spending by Category', 
    hover_data={'amt': ':.2f'},
    labels={'amt': 'Total Spend'}
)

# Customize the chart with percentages on the slices and display total spend on hover
fig.update_traces(
    textposition='inside', 
    textinfo='percent+label', 
    hovertemplate='Total Spend: $%{value:,.2f}', 
    pull=[0.05] * len(seniors_spending),  # Pull out each slice slightly
    marker=dict(line=dict(color='white', width=2))  # Add white border around each slice
)

# Update the layout for better alignment of the pie chart and legend
fig.update_layout(
    width=800,  # Adjust width
    height=500,  # Adjust height
    title={
        'text': 'Age: 79 & Older / Seniors / Elderly: Spending by Category', 
        'y':0.90,  # position of the title from bottom to top (0-1 scale)
        'x':0.56,   # center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'yanchor': 'middle',  # vertical centering
        'y': 0.5,             # legend is vertically centered
        'xanchor': 'left',     # move the legend to the right side
        'x': .8               # move the legend further right to avoid overlapping the pie chart
    },
    margin=dict(l=0)  # adds 100px left margin to move the pie chart to the left
)

# Display the chart
fig.show()


In [194]:
import plotly.express as px

# Step 1: Group data by 'age_description' and 'is_fraud'
fraud_by_age = df2.groupby('age_description')['is_fraud'].sum().reset_index()

# Step 2: Rename the columns for clarity
fraud_by_age.columns = ['Age Group', 'Fraudulent Transactions']

# Step 3: Calculate the total number of transactions for each age group to determine the percentage of fraudulent transactions
total_transactions_by_age = df2.groupby('age_description')['is_fraud'].count().reset_index()
total_transactions_by_age.columns = ['Age Group', 'Total Transactions']

# Step 4: Merge the dataframes to calculate the percentage of fraudulent transactions
fraud_comparison = pd.merge(fraud_by_age, total_transactions_by_age, on='Age Group')
fraud_comparison['Fraud Percentage'] = (fraud_comparison['Fraudulent Transactions'] / fraud_comparison['Total Transactions']) * 100

# Step 5: Create a bar chart using plotly for fraudulent transactions by age group
fig = px.bar(
    fraud_comparison, 
    x='Age Group', 
    y='Fraudulent Transactions', 
    title="Total Fraudulent Transactions by Age Group",
    labels={'Fraudulent Transactions': 'Fraud Cases', 'Age Group': 'Age Group'},
    height=500, width=800
)

# Display the chart
fig.show()

# Pie chart for the percentage of fraudulent transactions by age group
fig_pie = px.pie(
    fraud_comparison, 
    names='Age Group', 
    values='Fraud Percentage', 
    title="Percentage of Fraudulent Transactions by Age Group",
    height=500, width=800,
    hole=0.4
)

# Update the layout to center the legend vertically on the right margin and keep the pie chart size
fig_pie.update_traces(
    pull=[0.05 for _ in range(len(fraud_comparison))]  # Optional: slight "pull" effect for better clarity
)

# Adjust pie chart size by limiting the domain to avoid growth
fig_pie.update_traces(domain=dict(x=[0.0, 0.65]))  # Adjust the domain for pie chart width

fig_pie.update_layout(
    legend=dict(
        orientation="v",  # Keep the legend vertical
        yanchor="middle",  # Align the legend to the middle vertically
        y=0.5,  # Vertical center of the legend relative to the chart
        xanchor="left",  # Align the legend to the right of the chart
        x=0.9  # Position slightly to the right of the pie chart
    ),
    title={
        'text': 'Percentage of Fraudulent Transactions by Age Group',
        'y':0.90,  # Position the title closer to the top
        'x':0.5,  # Center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    margin=dict(t=50, l=50, r=0)  # Adjust margins for proper spacing
)

# Display the updated pie chart
fig_pie.show()



In [220]:
import plotly.express as px

# Step 1: Filter for fraudulent transactions
fraud_data = df2[df2['is_fraud'] == 1]

# Step 2: Group the data by 'category' and sum the 'amt' for each category
fraud_by_category = fraud_data.groupby('category')['amt'].sum().reset_index()

# Step 3: Calculate the total amount for each category (including non-fraudulent transactions)
total_by_category = df2.groupby('category')['amt'].sum().reset_index()

# Step 4: Merge both dataframes to calculate the percentage of fraud for each category
fraud_percentage_by_category = pd.merge(fraud_by_category, total_by_category, on='category', suffixes=('_fraud', '_total'))

# Step 5: Calculate the percentage of fraudulent transactions for each category
fraud_percentage_by_category['fraud_percentage'] = (fraud_percentage_by_category['amt_fraud'] / fraud_percentage_by_category['amt_total']) * 100

# Step 6: Create a pie chart using plotly for the percentage of fraudulent transactions by category
fig = px.pie(
    fraud_percentage_by_category, 
    values='fraud_percentage', 
    names='category', 
    title="Percentage of Fraudulent Transactions by Category",
    height=500, width=800, hole=0.40
)

# Step 7: Break out slices with a border and update the hover template
fig.update_traces(
    pull=[0.05 for _ in fraud_percentage_by_category['category']],  # Slightly pull out each slice
    textposition='inside', 
    textinfo='percent+label',
    hovertemplate='Category: %{label}<br>Fraud Percentage: %{value:.1f}%'  # Show percentage on hover
)

# Step 8: Match the layout and style to Cell 1
fig.update_layout(
    title={
        'text': "Percentage of Fraudulent Transactions by Category",
        'y':0.95,  # Position the title closer to the top
        'x':0.5,   # Center the title horizontally
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend=dict(
        orientation="v",  # Keep the legend vertical
        yanchor="middle",  # Align the legend to the middle vertically
        y=0.5,  # Vertical center of the legend relative to the chart
        xanchor="left",  # Align the legend to the right of the chart
        x=0.7  # Position slightly to the right of the pie chart
    ),
    margin=dict(t=50, l=50, r=0),  # Adjust margins to match Cell 1
    showlegend=True  # Ensure the legend is displayed
)

# Step 9: Adjust pie chart size by limiting the domain to avoid growth (optional step)
fig.update_traces(domain=dict(x=[0.0, 0.65]))

# Display the pie chart
fig.show()

